In [0]:
import time
import copy
import pyspark as ps
from pyspark.sql import SparkSession
from operator import add
from functools import reduce
from pyspark import SparkContext as sc
import re  
from decimal import Decimal
from datetime import datetime

In [0]:
spark = SparkSession.builder.getOrCreate()
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)
sc=spark.sparkContext

conf = ps.SparkConf().setMaster('local').setAppName("deltaFD")
spark=SparkSession.builder.config(conf=conf).getOrCreate()

In [0]:
df = spark.read.format("csv").option("header", "true").load('/FileStore/tables/test.csv')
df = df.drop('_c0')
df.createOrReplaceTempView("toy")

toy_df = spark.sql("select int(sensor_id), int(location), double(lat), double(lon), timestamp(timestamp), double(pressure), double(temperature), double(humidity) from toy limit 10000")

schema = toy_df.columns

In [0]:
def generate_computational_graph(RHS, schema):
    """
    Output
    ----------
    A dictionary where
    key: level
    value: list of current level's candidates, candidates are in the format of set
    -----

    """
    computational_graph=dict()
    for level in range(3):
        #use brute force to generate candidates for each level
        computational_graph[level]=[]
        if level== 0:
            for attribute  in schema:
                if attribute !=RHS:
                    computational_graph[level].append(set([attribute]))

        else:
            for element1 in computational_graph[level-1]:
                for element2 in computational_graph[0]:
                    newelement = element1.union(element2)
                    if newelement not in computational_graph[level]:
                        if len(newelement)==level+1:
                            computational_graph[level].append(newelement)    

    return computational_graph
def get_candidates(level, computational_graph):
    return computational_graph[level]
def prune_graph(level,current_level_result,computational_graph):
    """
    Input
    -------
    current_level_result: (soft/delta) functional dependencies discovered by algorithm, data structure: a list of candidates where candidates are in the format of sets
    computational_graph: A dict where key:level value: list of current level's candidates, candidates are in the format of set

    Output
    -------
    A pruned computational graph
    """
    # Candidates are pruned because minimal FD are already discovered

    # prune candidates after this level by verifying whether the next level has previous level's candidates as subset
    new_computational_graph = copy.deepcopy(computational_graph)
    while level<2:
        level+=1
        for LHS in current_level_result:
            for candidate in computational_graph[level]:
                if LHS.issubset(candidate):
                    if candidate in new_computational_graph[level]:
                        new_computational_graph[level].remove(candidate)


    return new_computational_graph
def transform_res(FDs):
    """
    Parameters
    --------------
    FDs: a list of (soft/delta) functional dependencies, where elements are tuples(LHS,RHS), LHS is in the format of set

    Output
    ---------
    current_level_result: a dictionary where key: RHS value: a list of LHS where candidates are in the form of sets
    """

    current_level_result=dict()
    for (LHS,RHS) in FDs:
        if RHS not in current_level_result.keys():
            current_level_result[RHS]=[]

        current_level_result[RHS].append(LHS)

    return current_level_result

In [0]:
def controller(df, func):
  """
  A control flow function

  Parameters
  -----------
  func: (soft/delta) Functional Discovery functions
  df: dataframe
  
  Output
  ------
  (soft/delta) Functional Dependencies
  """  
  # Initialization: Generate computational graph for each attribute which will be on RHS
  schema = df.columns
  computational_graph=dict()
  FDs=[]
  for RHS in schema:
    computational_graph[RHS]=generate_computational_graph(RHS,schema)

  for level in range(3):
    # Get current level candidates
    current_level_candidates=dict()
    for RHS in computational_graph.keys():
      current_level_candidates[RHS] = get_candidates(level,computational_graph[RHS])
    
#     print('candidates:',current_level_candidates)
    # Use current_level candidates as an input to FD-functions for each level, func will return discovered (soft/delta)functional dependencies
    tFDs = func(level,df,current_level_candidates)
#     print('FDs:',tFDs)
#     print(tFDs)
    FDs.extend(tFDs)
    #Transform res into a dictionary where key: RHS value: a list of LHS where candidates are in the form of sets
    current_level_result = transform_res(tFDs)
#     print(current_level_result)
    
    # Prune graphs according to feedback of FD-functions
#     print(f"level:{level}, computatioanl_graph_key:{computational_graph.keys()},current_level_result_key:{current_level_result.keys()}")
    for RHS in computational_graph.keys():
      if RHS in current_level_result.keys():
        computational_graph[RHS]=prune_graph(level, current_level_result[RHS],computational_graph[RHS])
    
  
  return FDs

In [0]:
def find_FDs_pairs(level,df,current_level_candidates):
  """
  Parameters
  -------------
  df: dataframe
  current_level_candidates: A dictionary where key:RHS value: a list of LHS, LHS are in a set format
  
  Output
  ---------
  A list of discovered functional dependencies where elements are tuples(LHS,RHS)
  """
  
  schema = df.columns
  FDs=[]
  
  #transform dataframe to rdds
  rdds = spark.sparkContext.emptyRDD()  
  for RHS in current_level_candidates.keys():
        for LHS in current_level_candidates[RHS]:
            FDs.append((LHS, RHS))
            pairs = df.rdd.map(lambda x:(((*LHS,RHS),*[x[schema.index(attribute)] for attribute in LHS]),{x[schema.index(RHS)]}))
            rdds = rdds.union(pairs)      
     
  level+=1

  rdds = rdds.coalesce(10).reduceByKey(lambda x,y: x.union(y))\
                .map(lambda x: (x[0][0], len(x[1]) == 1))\
                .filter(lambda x: x[1] == False)\
                .map(lambda x: x[0])\
                .distinct()\
                .collect()

  if len(rdds) != 0:
        for item in rdds:
            FDs.remove(({*item[:-1]},item[-1]))
    
  return FDs

In [0]:
RHS = 'lon'
LHS = {'lat'}
# delta = 0.1

# def for_map(x, RHS, LHS):
   
  
pairs = toy_df.rdd.map(lambda x:(((*LHS,RHS),*[x[schema.index(attribute)] for attribute in LHS]),{x[schema.index(RHS)]}))\
         .reduceByKey(lambda x,y: x.union(y))\
         .map(lambda x: (x[0][0], len(x[1]) == 1))\
         .filter(lambda x: x[1] == False)\
         .map(lambda x: x[0])\
         .distinct()\
         .collect()

#          .filter(lambda x: x[1] == False)\
#          .map(lambda x: x[0])\
#          .distinct()\
#          .collect()
#         .map(lambda x: (x[0][0], (max(x[1]) - min(x[1])) < delta))\
#         .filter(lambda x: x[1] == False)\
#         .map(lambda x: x[0])\
#         .distinct()\
#         .collect()
# #         .map(lambda x: ({x[0]}, 0))\
# #         .reduceByKey(add)\
# #         .collect()

print(pairs)


[('lat', 'lon')]

In [0]:
start_time = time.time()
FDs = controller(toy_df,find_FDs_pairs)
time_usage = time.time() - start_time
print("--- %s seconds ---" % (time_usage))
print(FDs)
print(len(FDs))

--- 131.5198268890381 seconds ---
[({'location'}, 'sensor_id'), ({'sensor_id'}, 'location'), ({'sensor_id'}, 'lat'), ({'location'}, 'lat'), ({'sensor_id'}, 'lon'), ({'location'}, 'lon'), ({'timestamp', 'pressure'}, 'sensor_id'), ({'humidity', 'pressure'}, 'sensor_id'), ({'timestamp', 'pressure'}, 'location'), ({'humidity', 'pressure'}, 'location'), ({'timestamp', 'pressure'}, 'lat'), ({'humidity', 'pressure'}, 'lat'), ({'timestamp', 'pressure'}, 'lon'), ({'humidity', 'pressure'}, 'lon'), ({'sensor_id', 'timestamp'}, 'pressure'), ({'timestamp', 'location'}, 'pressure'), ({'sensor_id', 'timestamp'}, 'temperature'), ({'timestamp', 'location'}, 'temperature'), ({'timestamp', 'pressure'}, 'temperature'), ({'sensor_id', 'timestamp'}, 'humidity'), ({'timestamp', 'location'}, 'humidity'), ({'timestamp', 'pressure'}, 'humidity'), ({'lon', 'timestamp', 'lat'}, 'sensor_id'), ({'lon', 'lat', 'pressure'}, 'sensor_id'), ({'timestamp', 'lat', 'temperature'}, 'sensor_id'), ({'lat', 'pressure', 'temperature'}, 'sensor_id'), ({'humidity', 'lat', 'temperature'}, 'sensor_id'), ({'lon', 'timestamp', 'temperature'}, 'sensor_id'), ({'lon', 'timestamp', 'humidity'}, 'sensor_id'), ({'lon', 'temperature', 'pressure'}, 'sensor_id'), ({'lon', 'humidity', 'temperature'}, 'sensor_id'), ({'timestamp', 'temperature', 'humidity'}, 'sensor_id'), ({'lon', 'timestamp', 'lat'}, 'location'), ({'lon', 'lat', 'pressure'}, 'location'), ({'timestamp', 'lat', 'temperature'}, 'location'), ({'lat', 'pressure', 'temperature'}, 'location'), ({'humidity', 'lat', 'temperature'}, 'location'), ({'lon', 'timestamp', 'temperature'}, 'location'), ({'lon', 'timestamp', 'humidity'}, 'location'), ({'lon', 'temperature', 'pressure'}, 'location'), ({'lon', 'humidity', 'temperature'}, 'location'), ({'timestamp', 'temperature', 'humidity'}, 'location'), ({'lon', 'timestamp', 'temperature'}, 'lat'), ({'lon', 'timestamp', 'humidity'}, 'lat'), ({'lon', 'temperature', 'pressure'}, 'lat'), ({'lon', 'humidity', 'temperature'}, 'lat'), ({'timestamp', 'temperature', 'humidity'}, 'lat'), ({'timestamp', 'lat', 'temperature'}, 'lon'), ({'lat', 'pressure', 'temperature'}, 'lon'), ({'humidity', 'lat', 'temperature'}, 'lon'), ({'timestamp', 'temperature', 'humidity'}, 'lon'), ({'lon', 'timestamp', 'lat'}, 'pressure'), ({'timestamp', 'lat', 'temperature'}, 'pressure'), ({'lon', 'timestamp', 'temperature'}, 'pressure'), ({'lon', 'timestamp', 'humidity'}, 'pressure'), ({'timestamp', 'temperature', 'humidity'}, 'pressure'), ({'lon', 'timestamp', 'lat'}, 'temperature'), ({'lon', 'timestamp', 'humidity'}, 'temperature'), ({'lon', 'timestamp', 'lat'}, 'humidity'), ({'timestamp', 'lat', 'temperature'}, 'humidity'), ({'lon', 'timestamp', 'temperature'}, 'humidity')]
61

In [0]:
start_time = time.time()
FDs = controller(toy_df,find_FDs_pairs)
time_usage = time.time() - start_time
print("--- %s seconds ---" % (time_usage))
print(FDs)
print(len(FDs))

--- 432.81809186935425 seconds ---
[({'sensor_id'}, 'location'), ({'sensor_id'}, 'lon'), ({'location'}, 'sensor_id'), ({'location'}, 'lon'), ({'location'}, 'lat'), ({'sensor_id'}, 'lat'), ({'timestamp', 'pressure'}, 'location'), ({'sensor_id', 'timestamp'}, 'humidity'), ({'timestamp', 'location'}, 'temperature'), ({'sensor_id', 'timestamp'}, 'pressure'), ({'humidity', 'pressure'}, 'lat'), ({'timestamp', 'pressure'}, 'temperature'), ({'timestamp', 'location'}, 'humidity'), ({'timestamp', 'pressure'}, 'lon'), ({'timestamp', 'pressure'}, 'sensor_id'), ({'sensor_id', 'timestamp'}, 'temperature'), ({'timestamp', 'pressure'}, 'humidity'), ({'humidity', 'pressure'}, 'lon'), ({'timestamp', 'pressure'}, 'lat'), ({'humidity', 'pressure'}, 'sensor_id'), ({'humidity', 'pressure'}, 'location'), ({'timestamp', 'location'}, 'pressure'), ({'humidity', 'lat', 'temperature'}, 'lon'), ({'lon', 'timestamp', 'lat'}, 'sensor_id'), ({'timestamp', 'lat', 'temperature'}, 'location'), ({'lon', 'timestamp', 'humidity'}, 'lat'), ({'lon', 'temperature', 'pressure'}, 'location'), ({'lon', 'humidity', 'temperature'}, 'location'), ({'timestamp', 'lat', 'temperature'}, 'humidity'), ({'lon', 'timestamp', 'temperature'}, 'humidity'), ({'lat', 'pressure', 'temperature'}, 'lon'), ({'timestamp', 'temperature', 'humidity'}, 'sensor_id'), ({'timestamp', 'lat', 'temperature'}, 'sensor_id'), ({'lon', 'timestamp', 'humidity'}, 'location'), ({'timestamp', 'lat', 'temperature'}, 'lon'), ({'timestamp', 'temperature', 'humidity'}, 'location'), ({'lon', 'temperature', 'pressure'}, 'lat'), ({'lon', 'timestamp', 'temperature'}, 'pressure'), ({'lon', 'lat', 'pressure'}, 'sensor_id'), ({'lon', 'timestamp', 'humidity'}, 'pressure'), ({'lat', 'pressure', 'temperature'}, 'location'), ({'lon', 'timestamp', 'lat'}, 'location'), ({'lon', 'timestamp', 'lat'}, 'pressure'), ({'timestamp', 'temperature', 'humidity'}, 'lat'), ({'timestamp', 'temperature', 'humidity'}, 'lon'), ({'lon', 'timestamp', 'temperature'}, 'sensor_id'), ({'lon', 'timestamp', 'humidity'}, 'sensor_id'), ({'humidity', 'lat', 'temperature'}, 'sensor_id'), ({'lon', 'timestamp', 'humidity'}, 'temperature'), ({'lon', 'humidity', 'temperature'}, 'sensor_id'), ({'lon', 'timestamp', 'lat'}, 'humidity'), ({'lon', 'lat', 'pressure'}, 'location'), ({'lat', 'pressure', 'temperature'}, 'sensor_id'), ({'timestamp', 'temperature', 'humidity'}, 'pressure'), ({'lon', 'timestamp', 'lat'}, 'temperature'), ({'lon', 'timestamp', 'temperature'}, 'location'), ({'lon', 'timestamp', 'temperature'}, 'lat'), ({'lon', 'temperature', 'pressure'}, 'sensor_id'), ({'lon', 'humidity', 'temperature'}, 'lat'), ({'timestamp', 'lat', 'temperature'}, 'pressure'), ({'humidity', 'lat', 'temperature'}, 'location')]
61

In [0]:
current_level_candidates = dict()
for RHS in schema:
  current_level_candidates[RHS] = generate_computational_graph(RHS, schema)[0]
  
df = toy_df